In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# importing the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,cohen_kappa_score,roc_auc_score,roc_curve
from sklearn.metrics import precision_score,recall_score,plot_confusion_matrix,plot_roc_curve,f1_score
from sklearn.feature_selection import SequentialFeatureSelector as sfs,RFE
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize']=[15,7]

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
df = pd.read_csv('/kaggle/input/customer-analytics/Train.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df = df.drop('ID',axis=1)
df.head()

In [ ]:
# Categorical data

cat = df.select_dtypes(include='object').columns
len(cat)

In [ ]:
r=3
c=2
it=1
for i in cat:
    plt.subplot(r,c,it)
    sns.countplot(x=df[i])
    plt.title(i)
    it+=1
plt.tight_layout()
plt.show()

In [ ]:
# Numerical data

num = df.select_dtypes(include=np.number).columns
len(num)

In [ ]:
r=4
c=2
it=1
for i in num:
    plt.subplot(r,c,it)
    sns.distplot(df[i],hist=False)
    plt.title(i)
    plt.grid()
    it+=1
plt.tight_layout()
plt.show()

In [ ]:
# outliers

df.boxplot()
plt.show()

# Encoding and Scaling

In [ ]:
y=df['Reached.on.Time_Y.N']
x=df.drop('Reached.on.Time_Y.N',axis=1)

In [ ]:
# Encoding the categorical columns:

x_cat = x.select_dtypes(include='object')
x_cat.head()

In [ ]:
x_cat=pd.get_dummies(x_cat,drop_first=True)
x_cat.head()

In [ ]:
# Scaling the numerical columns

x_num = x.select_dtypes(include=np.number)
x_num.head()

In [ ]:
ss = StandardScaler()
x_num = pd.DataFrame(ss.fit_transform(x_num),columns=x_num.columns)
x_num.head()

In [ ]:
x = pd.concat([x_cat,x_num],axis=1)
x.head()

# Base Model

In [ ]:
xc = sm.add_constant(x)

In [ ]:
model = sm.Logit(y,xc).fit()
print(model.summary())

In [ ]:
xc=sm.add_constant(x)
cols=list(xc.columns)
while len(cols)>1:
    xc=xc[cols]
    model=sm.Logit(y,xc).fit()
    p=model.pvalues
    pmax=max(p)
    pid=p.idxmax()           # index of series with highest value
    if pmax>0.05:
        cols.remove(pid)
        print(pid,pmax)
    else:
        break
cols

In [ ]:
xc=xc[['const',
 'Product_importance_low',
 'Product_importance_medium',
 'Customer_care_calls',
 'Cost_of_the_Product',
 'Prior_purchases',
 'Discount_offered',
 'Weight_in_gms']]

In [ ]:
xc.head()

In [ ]:
model = sm.Logit(y,xc).fit()
print(model.summary())

In [ ]:
# significant columns are:

# 'Product_importance_low','Product_importance_medium','Customer_care_calls',
# 'Cost_of_the_Product','Prior_purchases','Discount_offered','Weight_in_gms'

# Building other models


In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=42)
print("The shape of xtrain is: ",xtrain.shape)
print("The shape of xtest is: ",xtest.shape)
print("The shape of ytrain is: ",ytrain.shape)
print("The shape of ytest is: ",ytest.shape)

In [ ]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
knn = KNeighborsClassifier()
gnb = GaussianNB()

model_list = [lr,dt,rf,knn,gnb]

train_acc = []
test_acc = []

for i in model_list:
    i_model = i.fit(xtrain,ytrain)
    ytrain_pred = i_model.predict(xtrain)
    ytest_pred = i_model.predict(xtest)
    train_acc.append(accuracy_score(ytrain,ytrain_pred))
    test_acc.append(accuracy_score(ytest,ytest_pred))

In [ ]:
# visualizing the accuracy of models

models = ['Logistic Regression','Decision Tree Classifier','Random Forest Classifier','KNeighbors Classifier','Gaussian Naive Bayes']

In [ ]:
# visualizing the train model accuracy 

sns.barplot(y=models,x=train_acc)
plt.show()


In [ ]:
# visualizing the test model accuracy 


sns.barplot(y=models,x=test_acc)
plt.show()


# Confusion Matrix

In [ ]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
knn = KNeighborsClassifier()
gnb = GaussianNB()

model_list = [lr,dt,rf,knn,gnb]


r=3
c=2
it=1
for i in model_list:
    i_model = i.fit(xtrain,ytrain)
    ytrain_pred = i_model.predict(xtrain)
    ytest_pred = i_model.predict(xtest)
    plt.subplot(r,c,it)
    sns.heatmap(confusion_matrix(ytrain,ytrain_pred),annot=True,cmap='Blues')
    plt.title(i)
    it+=1
plt.tight_layout()
plt.show()

In [ ]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
knn = KNeighborsClassifier()
gnb = GaussianNB()

model_list = [lr,dt,rf,knn,gnb]


r=3
c=2
it=1
for i in model_list:
    i_model = i.fit(xtrain,ytrain)
    ytrain_pred = i_model.predict(xtrain)
    ytest_pred = i_model.predict(xtest)
    plt.subplot(r,c,it)
    sns.heatmap(confusion_matrix(ytest,ytest_pred),annot=True,cmap='Blues')
    plt.title(i)
    it+=1
plt.tight_layout()
plt.show()

# Comparing the accuracies of the train and test sets¶


In [ ]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
knn = KNeighborsClassifier()
gnb = GaussianNB()


model_dict = {'Model':model_list,'Train Score':train_acc,'Test Score':test_acc}
model_dict

In [ ]:
model_summary = pd.DataFrame({'Model':model_list,'Train Score':train_acc,'Test Score':test_acc})
model_summary

# Finding the best parameters¶


**Decision Tree**

In [ ]:
dt = DecisionTreeClassifier()
params=[{'criterion':['gini','entropy'],
             'max_depth':range(2,5),
             'min_samples_leaf':range(2,7),
             }]

grd = GridSearchCV(estimator=dt,param_grid=params,cv=3)

grd_model = grd.fit(xtrain,ytrain)

grd_model.best_params_

In [ ]:
dt = DecisionTreeClassifier(criterion=grd_model.best_params_.get('criterion'),
                           max_depth=grd_model.best_params_.get('max_depth'),
                           min_samples_leaf=grd_model.best_params_.get('max_depth'))

dt_model = dt.fit(xtrain,ytrain)

ytrain_pred = dt_model.predict(xtrain)
ytest_pred = dt_model.predict(xtest)

In [ ]:
print("The accuracy score of regularized Decision tree is: ",accuracy_score(ytrain,ytrain_pred))
print("The accuracy score of regularized Decision tree is: ",accuracy_score(ytest,ytest_pred))

**Random Forest**

In [ ]:
rf = RandomForestClassifier()
params=[{'n_estimators':[100,150,200],
       'criterion':['gini','entropy'],
             'max_depth':range(2,5),
             'min_samples_leaf':range(2,7),
             }]

grd = GridSearchCV(estimator=rf,param_grid=params,cv=3)

grd_model = grd.fit(xtrain,ytrain)

grd_model.best_params_

In [ ]:
rf = RandomForestClassifier(criterion=grd_model.best_params_.get('criterion'),
                           max_depth=grd_model.best_params_.get('max_depth'),
                           min_samples_leaf=grd_model.best_params_.get('max_depth'),
                           n_estimators=grd_model.best_params_.get('n_estimators'))

rf_model = rf.fit(xtrain,ytrain)

ytrain_pred = rf_model.predict(xtrain)
ytest_pred = rf_model.predict(xtest)

In [ ]:
print("The accuracy score of regularized Random Forest is: ",accuracy_score(ytrain,ytrain_pred))
print("The accuracy score of regularized Random Forest is: ",accuracy_score(ytest,ytest_pred))

**KNeignbors Classifier**



In [ ]:
knn = KNeighborsClassifier()
params=[{'n_neighbors':[3,5,7],
         'metric':['euclidean','manhattan']}]

grd = GridSearchCV(estimator=knn,param_grid=params,cv=3)

grd_model = grd.fit(xtrain,ytrain)

grd_model.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors=grd_model.best_params_.get('n_neighbors'),
                          metric=grd_model.best_params_.get('metric'))

knn_model = knn.fit(xtrain,ytrain)

ytrain_pred = knn_model.predict(xtrain)
ytest_pred = knn_model.predict(xtest)

In [ ]:
print("The accuracy score of regularized KN classifier is: ",accuracy_score(ytrain,ytrain_pred))
print("The accuracy score of regularized KN Classifier is: ",accuracy_score(ytest,ytest_pred))

**Thus we can see that the Decision Tree and Random Forest are giving good results after subjected to GridSearchCV**



**Boosting**

In [ ]:
# GradientBoosting

gbm = GradientBoostingClassifier(learning_rate=0.1,n_estimators=100,random_state=10)
gbm_model = gbm.fit(xtrain,ytrain)
ytrain_pred = gbm_model.predict(xtrain)
ytest_pred = gbm_model.predict(xtest)
print(accuracy_score(ytrain,ytrain_pred),";",accuracy_score(ytest,ytest_pred))

In [ ]:
# XGBoosting

xgb = XGBClassifier(n_estimators=100,max_depth=2)
xgb.fit(xtrain,ytrain)
ytrain_pred = xgb.predict(xtrain)
ytest_pred = xgb.predict(xtest)
print(accuracy_score(ytrain,ytrain_pred),";",accuracy_score(ytest,ytest_pred))

# Feature Importances

In [ ]:
rf_model = rf.fit(xtrain,ytrain)
rf_model.feature_importances_

In [ ]:
importances = pd.DataFrame(xtrain.columns,columns=['Features'])
importances

In [ ]:
importances['Feature_importances'] = rf_model.feature_importances_
importances

In [ ]:
imp = importances.sort_values(by='Feature_importances',ascending=False)
imp

In [ ]:
sns.barplot(y=imp['Features'],x=imp['Feature_importances'])
plt.show()

In [ ]:
# Important Features according to above mentioned Barplot:

imp['Features'][0:7].values

**'Discount_offered', 'Weight_in_gms', 'Prior_purchases',
       'Cost_of_the_Product', 'Customer_care_calls', 'Customer_rating',
       'Gender_M'**
are some of the important features which impact the E-Commerce shippings
